In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [17]:
from scipy.io import arff

arff_file = arff.loadarff('/content/KDDTrain+.arff')
df = pd.DataFrame(arff_file[0])


In [ ]:
print(df.dtypes)

In [ ]:
df

In [ ]:
df.columns = df.columns.astype(str)


X = df.iloc[:, :-1]
y = df.iloc[:, -1]

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X = pd.get_dummies(X, drop_first=True)

In [ ]:
X

In [ ]:
y

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
class Classifier(nn.Module):
	def __init__(self, input_size):
		super(Classifier, self).__init__()
		self.fc1 = nn.Linear(input_size, 128)
		self.fc2 = nn.Linear(128, 64)
		self.fc3 = nn.Linear(64, 2)
		self.relu = nn.ReLU()
		self.dropout = nn.Dropout(0.3)

	def forward(self, x):
		x = self.relu(self.fc1(x))
		x = self.dropout(x)
		x = self.relu(self.fc2(x))
		x = self.dropout(x)
		x = self.fc3(x)
		return x

input_size = X.shape[1]
model = Classifier(input_size)


In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
	model.train()
	total_loss = 0
	correct = 0
	total = 0

	for inputs, labels in train_loader:
		optimizer.zero_grad()
		outputs = model(inputs)
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()

		total_loss += loss.item()
		_, predicted = torch.max(outputs, 1)
		total += labels.size(0)
		correct += (predicted == labels).sum().item()

	accuracy = 100 * correct / total
	print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%')
